# Python `__slots__`
* 참고: https://www.chrisbarra.xyz/posts/let-me-introduce-slots/

* `getsizeof`를 활용하여 객체의 메모리 점유 크기 파악

In [1]:
from sys import getsizeof

In [2]:
class SimpleClass():
    def __init__(self, message):
        self.message = message
        self.capital_message = self.make_it_bigger()
        
    def make_it_bigger(self):
        return self.message.upper()
    
    def scream_message(self):
        print(self.capital_message)

In [3]:
my_instance = SimpleClass("My Message")

### 파이썬이 사용하는 특별한 속성: `__dict__`
* 파이썬에서 인스턴스의 속성을 저장하는 공간
* `instanace.attribute_name` 또는 `instance.__dict__['attribute_name']` 형식으로 접근

In [5]:
dir(my_instance)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'capital_message',
 'make_it_bigger',
 'message',
 'scream_message']

In [4]:
[element for element in dir(my_instance) if element == '__dict__']

['__dict__']

In [6]:
my_instance.__dict__

{'capital_message': 'MY MESSAGE', 'message': 'My Message'}

In [7]:
my_instance.new_message = "New Message"
my_instance.__dict__

{'capital_message': 'MY MESSAGE',
 'message': 'My Message',
 'new_message': 'New Message'}

In [8]:
my_instance.__dict__['another_message'] = "Yet Another Message"
my_instance.__dict__

{'another_message': 'Yet Another Message',
 'capital_message': 'MY MESSAGE',
 'message': 'My Message',
 'new_message': 'New Message'}

### 저장소로서의 `dict`
* access time = $O(1)$ : 즉 dictionary의 크기와 무관

In [9]:
class ClassWithSlots():
    __slots__ = ["message", "capital_message"]   # 이 부분만 다름
    
    def __init__(self, message):
        self.message = message
        self.capital_message = self.make_it_bigger()
        
    def make_it_bigger(self):
        return self.message.upper()
    
    def scream_message(self):
        print(self.capital_message)

In [10]:
my_instance = ClassWithSlots("My Message")

In [11]:
# 이전과 결과 다름
[element for element in dir(my_instance) if element == '__dict__']

[]

In [12]:
# __dict__ 없는 대신 __slots__ 존재
[element for element in dir(my_instance) if element == '__slots__']

['__slots__']

In [13]:
my_instance.message

'My Message'

In [14]:
my_instance.capital_message

'MY MESSAGE'

In [15]:
# 이전과 달리 오류
my_instance.new_message = "New Message"

AttributeError: 'ClassWithSlots' object has no attribute 'new_message'

In [16]:
# 이미 정의되어 있는 변수는 변경 가능
my_instance.message = "My Replaced Message"

### `__dict__` 메모리 점유

In [27]:
my_instance_without_slots = SimpleClass("My Message")
my_instance_with_slots = ClassWithSlots("My Message")

In [28]:
getsizeof(my_instance_without_slots)  # bytes

56

In [29]:
getsizeof(my_instance_with_slots)   # bytes

56

* 클래스의 `getsizeof`는 참조하고 있는 객체들을 반영하지 못함

In [30]:
my_instance_without_slots.__dict__

{'capital_message': 'MY MESSAGE', 'message': 'My Message'}

In [31]:
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(128, 56)

In [34]:
my_instance_without_slots.new_attribute_1 = "This is a new attribute 1"
my_instance_without_slots.new_attribute_2 = "This is a new attribute 2"
my_instance_without_slots.new_attribute_3 = "This is a new attribute 3"
my_instance_without_slots.__dict__

{'capital_message': 'MY MESSAGE',
 'message': 'My Message',
 'new_attribute_1': 'This is a new attribute 1',
 'new_attribute_2': 'This is a new attribute 2',
 'new_attribute_3': 'This is a new attribute 3'}

In [35]:
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(480, 56)

* `__dict__`의 크기가 변함

In [41]:
for i in range(10):
    my_instance_without_slots.__dict__[i] = str(i)
    
my_instance_without_slots.__dict__

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 'capital_message': 'MY MESSAGE',
 'message': 'My Message',
 'new_attribute_1': 'This is a new attribute 1',
 'new_attribute_2': 'This is a new attribute 2',
 'new_attribute_3': 'This is a new attribute 3'}

In [42]:
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(864, 56)

### 일반 클래스와 달리 `__slots__`를 사용한 경우

In [43]:
import json

my_json = '''{
    "username": "use@python3.org",
    "country": "Poland", "website":
    "www.chrisbarra.xzy",
    "date": "2017/08/15",
    "uid": 1, "gender": "Male"
}'''

In [44]:
class MyUserWithSlots():
    """슬롯을 가진 클래스"""
    
    __slots__ = ('username', 'country', 'website', 'date')
    
    def __init__ (self, username, country, website, date, **kwargs):
        self.username = username
        self.country = country
        self.website = website
        self.date = date
        

class MyUserWithoutSlots():
    """슬롯을 사용하지 않은 일반 클래스"""
    
    def __init__ (self, username, country, website, date, **kwargs):
        self.username = username
        self.country = country
        self.website = website
        self.date = date

In [45]:
def get_size(instance):
    """__dict__가 있는 경우, 그 크기를 추가 계산"""
    
    size_dict = 0
    
    try:
        size_dict = getsizeof(instance.__dict__)
    except AttributeError:
        pass
    
    return size_dict + getsizeof(instance)

In [46]:
# 1,000,000개 인스턴스 생성할 것임
NUM_INSTANCES = 1000000

In [47]:
# 슬롯 있는 클래스로 인스턴스들을 만들고 각 크기들을 리스트에 담음
with_slots = [get_size(MyUserWithSlots(**json.loads(my_json))) 
              for _ in range(NUM_INSTANCES)]

# 리스트 내 값들을 모두 더함
size_with_slots = sum(with_slots)/1000000

print("전체 크기는 {} MB".format(size_with_slots))

전체 크기는 72.0 MB


In [48]:
# 슬롯 없는 클래스로 인스턴스들을 만들고 각 크기들을 리스트에 담음
without_slots = [get_size(MyUserWithoutSlots(**json.loads(my_json))) 
                 for _ in range(NUM_INSTANCES)]

# 리스트 내 값들을 모두 더함
size_without_slots = sum(without_slots)/1000000

print("전체 크기는 {} MB".format(size_without_slots))

전체 크기는 248.0 MB


### 접근 시간 비교
* `__slots__`를 사용한 경우가 약간 빠름

In [49]:
instance_with_slots = MyUserWithSlots(**json.loads(my_json))

In [50]:
%%timeit
z = instance_with_slots.username

70.4 ns ± 1.53 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [51]:
instance_without_slots = MyUserWithoutSlots(**json.loads(my_json))

In [52]:
%%timeit
z = instance_without_slots.username

76.8 ns ± 4.07 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
